In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, current_timestamp
import psycopg2
import logging
import sys

# ==========================
# Configuration and Setup
# ==========================

# Configure Logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

# Create SparkSession
spark = SparkSession.builder \
    .appName("ETL Process with OVERRIDING SYSTEM VALUE") \
    .config("spark.master", "local[*]") \
    .config("spark.jars", "/opt/spark/jars/postgresql-42.6.0.jar") \
    .getOrCreate()

logger.info("SparkSession created successfully.")

# Database Configuration
jdbc_url = "jdbc:postgresql://172.27.253.185:5430/postgres_db"
db_properties = {
    "user": "postgres_user",
    "password": "postgres_password",
    "driver": "org.postgresql.Driver"
}

# PostgreSQL Connection Parameters for psycopg2
pg_conn_params = {
    "dbname": "postgres_db",
    "user": "postgres_user",
    "password": "postgres_password",
    "host": "172.27.253.185",
    "port": 5430
}

# ==========================
# Helper Functions
# ==========================

def upsert_with_overriding_system_value(source_df, target_table, conflict_column, exclude_columns=[]):
    """
    Performs an upsert operation on the target_table using data from source_df.
    Excludes specified columns from being updated to accommodate GENERATED ALWAYS constraints.
    
    Parameters:
    - source_df (DataFrame): Source Spark DataFrame to upsert.
    - target_table (str): Target table in the database.
    - conflict_column (str): Column to detect conflicts (usually primary key).
    - exclude_columns (list): List of columns to exclude from the UPDATE operation.
    """
    temp_table = f"{target_table}_temp"
    try:
        logger.info(f"Writing data to temporary table {temp_table}.")
        source_df.write \
            .format("jdbc") \
            .option("url", jdbc_url) \
            .option("dbtable", temp_table) \
            .option("user", db_properties["user"]) \
            .option("password", db_properties["password"]) \
            .option("driver", db_properties["driver"]) \
            .mode("overwrite") \
            .save()

        # Generate column lists with proper quoting to handle case sensitivity and reserved keywords
        source_columns = ", ".join([f'"{c}"' for c in source_df.columns])
        update_columns = [
            f'"{c}" = EXCLUDED."{c}"' for c in source_df.columns
            if c != conflict_column and c not in exclude_columns
        ]

        # Formulate SQL statement
        insert_sql = f"""
        INSERT INTO {target_table}
        ({source_columns})
        OVERRIDING SYSTEM VALUE
        SELECT {source_columns}
        FROM {temp_table}
        ON CONFLICT ({conflict_column})
        DO UPDATE
        SET {", ".join(update_columns)};
        """

        logger.info(f"Executing SQL: {insert_sql}")  # Added SQL logging

        # Connect to PostgreSQL and execute the upsert
        with psycopg2.connect(**pg_conn_params) as conn:
            with conn.cursor() as cursor:
                cursor.execute(insert_sql)
                conn.commit()
                logger.info(f"Upsert completed successfully for table {target_table}.")
    except Exception as e:
        logger.error(f"Error during upsert operation for table {target_table}: {e}")
        raise
    finally:
        # Optional: Drop the temporary table if no longer needed
        try:
            with psycopg2.connect(**pg_conn_params) as conn:
                with conn.cursor() as cursor:
                    cursor.execute(f'DROP TABLE IF EXISTS {temp_table};')
                    conn.commit()
                    logger.info(f"Temporary table {temp_table} dropped successfully.")
        except Exception as e:
            logger.warning(f"Could not drop temporary table {temp_table}: {e}")

def list_tables(schema):
    """
    Retrieves the list of table names in the specified schema.
    
    Parameters:
    - schema (str): The schema name.
    
    Returns:
    - list: List of table names.
    """
    query = f"""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = '{schema}';
    """
    try:
        with psycopg2.connect(**pg_conn_params) as conn:
            with conn.cursor() as cursor:
                cursor.execute(query)
                tables = cursor.fetchall()
                return [table[0] for table in tables]
    except Exception as e:
        logger.error(f"Error retrieving tables in schema {schema}: {e}")
        return []

# ==========================
# ETL Process
# ==========================

try:
    # Load data from ROW layer
    logger.info("Loading data from ROW layer.")

    # Define all row layer tables
    row_layer_tables = [
        "source1.craft_market_wide",
        "source2.craft_market_masters_products",
        "source2.craft_market_orders_customers",
        "source3.craft_market_craftsmans",
        "source3.craft_market_customers",
        "source3.craft_market_orders"
    ]

    # Verify existence of tables in their respective schemas
    for table in row_layer_tables:
        schema, table_name = table.split('.')
        existing_tables = list_tables(schema)
        if table_name not in existing_tables:
            logger.error(f"Table {table} does not exist in schema {schema}.")
            raise Exception(f"Table {table} does not exist in schema {schema}.")
        else:
            logger.info(f"Table {table} found in schema {schema}.")

    # Read all row layer tables into separate DataFrames
    row_layer_data = {}
    for table in row_layer_tables:
        try:
            logger.info(f"Loading data from {table}.")
            df = spark.read \
                .format("jdbc") \
                .option("url", jdbc_url) \
                .option("dbtable", table) \
                .option("user", db_properties["user"]) \
                .option("password", db_properties["password"]) \
                .option("driver", db_properties["driver"]) \
                .load()
            row_layer_data[table] = df
            logger.info(f"Data loaded from {table} successfully.")
        except Exception as table_e:
            logger.error(f"Failed to load data from {table}: {table_e}")
            raise  # Re-raise to trigger the outer exception handler

    # ==========================
    # Process source1.craft_market_wide
    # ==========================
    logger.info("Processing table source1.craft_market_wide.")
    source1_df = row_layer_data["source1.craft_market_wide"]

    # Process d_craftsmans
    d_craftsmans = source1_df.select(
        col("craftsman_id").alias("craftsman_id"),
        col("craftsman_name").alias("craftsman_name"),
        col("craftsman_address").alias("craftsman_address"),
        col("craftsman_birthday").alias("craftsman_birthday"),
        col("craftsman_email").alias("craftsman_email"),
        current_timestamp().alias("load_dttm")
    ).distinct()

    upsert_with_overriding_system_value(
        source_df=d_craftsmans,
        target_table="dwh.d_craftsmans",
        conflict_column="craftsman_id"
    )

    # Process d_customers
    d_customers = source1_df.select(
        col("customer_id").alias("customer_id"),
        col("customer_name").alias("customer_name"),
        col("customer_address").alias("customer_address"),
        col("customer_birthday").alias("customer_birthday"),
        col("customer_email").alias("customer_email"),
        current_timestamp().alias("load_dttm")
    ).distinct()

    upsert_with_overriding_system_value(
        source_df=d_customers,
        target_table="dwh.d_customers",
        conflict_column="customer_id"
    )

    # Process d_products
    d_products = source1_df.select(
        col("product_id").alias("product_id"),
        col("product_name").alias("product_name"),
        col("product_description").alias("product_description"),
        col("product_type").alias("product_type"),
        col("product_price").alias("product_price"),
        current_timestamp().alias("load_dttm")
    ).distinct()

    upsert_with_overriding_system_value(
        source_df=d_products,
        target_table="dwh.d_products",
        conflict_column="product_id"
    )

    # Process f_orders
    f_orders = source1_df.select(
        col("order_id").alias("order_id"),
        col("product_id").alias("product_id"),  # GENERATED ALWAYS column
        col("craftsman_id").alias("craftsman_id"),
        col("customer_id").alias("customer_id"),
        col("order_created_date").alias("order_created_date"),
        col("order_completion_date").alias("order_completion_date"),
        col("order_status").alias("order_status"),
        current_timestamp().alias("load_dttm")
    )

    upsert_with_overriding_system_value(
        source_df=f_orders,
        target_table="dwh.f_orders",
        conflict_column="order_id",
        exclude_columns=["product_id"]  # Exclude product_id from updates
    )

    # ==========================
    # Process source2.craft_market_masters_products
    # ==========================
    logger.info("Processing table source2.craft_market_masters_products.")
    source2_df = row_layer_data["source2.craft_market_masters_products"]

    # Process d_craftsmans from source2
    d_craftsmans_s2 = source2_df.select(
        col("craftsman_id").alias("craftsman_id"),
        col("craftsman_name").alias("craftsman_name"),
        col("craftsman_address").alias("craftsman_address"),
        col("craftsman_birthday").alias("craftsman_birthday"),
        col("craftsman_email").alias("craftsman_email"),
        current_timestamp().alias("load_dttm")
    ).distinct()

    upsert_with_overriding_system_value(
        source_df=d_craftsmans_s2,
        target_table="dwh.d_craftsmans",
        conflict_column="craftsman_id"
    )

    # Process d_products from source2
    d_products_s2 = source2_df.select(
        col("product_id").alias("product_id"),
        col("product_name").alias("product_name"),
        col("product_description").alias("product_description"),
        col("product_type").alias("product_type"),
        col("product_price").alias("product_price"),
        current_timestamp().alias("load_dttm")
    ).distinct()

    upsert_with_overriding_system_value(
        source_df=d_products_s2,
        target_table="dwh.d_products",
        conflict_column="product_id"
    )

    # ==========================
    # Process source2.craft_market_orders_customers
    # ==========================
    logger.info("Processing table source2.craft_market_orders_customers.")
    source2_orders_customers_df = row_layer_data["source2.craft_market_orders_customers"]

    # Предполагаем, что эта таблица содержит информацию о заказах и клиентах
    # Если в ней содержатся данные, аналогичные source3.craft_market_orders и source3.craft_market_customers,
    # необходимо обработать их соответственно

    # Обработка d_customers из source2.craft_market_orders_customers
    d_customers_s2_orders = source2_orders_customers_df.select(
        col("customer_id").alias("customer_id"),
        col("customer_name").alias("customer_name"),
        col("customer_address").alias("customer_address"),
        col("customer_birthday").alias("customer_birthday"),
        col("customer_email").alias("customer_email"),
        current_timestamp().alias("load_dttm")
    ).distinct()

    upsert_with_overriding_system_value(
        source_df=d_customers_s2_orders,
        target_table="dwh.d_customers",
        conflict_column="customer_id"
    )

    # Обработка f_orders из source2.craft_market_orders_customers
    f_orders_s2_orders_customers = source2_orders_customers_df.select(
        col("order_id").alias("order_id"),
        col("product_id").alias("product_id"),  # GENERATED ALWAYS column
        col("craftsman_id").alias("craftsman_id"),
        col("customer_id").alias("customer_id"),
        col("order_created_date").alias("order_created_date"),
        col("order_completion_date").alias("order_completion_date"),
        col("order_status").alias("order_status"),
        current_timestamp().alias("load_dttm")
    )

    upsert_with_overriding_system_value(
        source_df=f_orders_s2_orders_customers,
        target_table="dwh.f_orders",
        conflict_column="order_id",
        exclude_columns=["product_id"]  # Exclude product_id from updates
    )

    # ==========================
    # Process source3.craft_market_craftsmans
    # ==========================
    logger.info("Processing table source3.craft_market_craftsmans.")
    source3_craftsmans_df = row_layer_data["source3.craft_market_craftsmans"]

    d_craftsmans_s3 = source3_craftsmans_df.select(
        col("craftsman_id").alias("craftsman_id"),
        col("craftsman_name").alias("craftsman_name"),
        col("craftsman_address").alias("craftsman_address"),
        col("craftsman_birthday").alias("craftsman_birthday"),
        col("craftsman_email").alias("craftsman_email"),
        current_timestamp().alias("load_dttm")
    ).distinct()

    upsert_with_overriding_system_value(
        source_df=d_craftsmans_s3,
        target_table="dwh.d_craftsmans",
        conflict_column="craftsman_id"
    )

    # ==========================
    # Process source3.craft_market_customers
    # ==========================
    logger.info("Processing table source3.craft_market_customers.")
    source3_customers_df = row_layer_data["source3.craft_market_customers"]

    d_customers_s3 = source3_customers_df.select(
        col("customer_id").alias("customer_id"),
        col("customer_name").alias("customer_name"),
        col("customer_address").alias("customer_address"),
        col("customer_birthday").alias("customer_birthday"),
        col("customer_email").alias("customer_email"),
        current_timestamp().alias("load_dttm")
    ).distinct()

    upsert_with_overriding_system_value(
        source_df=d_customers_s3,
        target_table="dwh.d_customers",
        conflict_column="customer_id"
    )

    # ==========================
    # Process source3.craft_market_orders
    # ==========================
    logger.info("Processing table source3.craft_market_orders.")
    source3_orders_df = row_layer_data["source3.craft_market_orders"]

    f_orders_s3 = source3_orders_df.select(
        col("order_id").alias("order_id"),
        col("product_id").alias("product_id"),  # GENERATED ALWAYS column
        col("craftsman_id").alias("craftsman_id"),
        col("customer_id").alias("customer_id"),
        col("order_created_date").alias("order_created_date"),
        col("order_completion_date").alias("order_completion_date"),
        col("order_status").alias("order_status"),
        current_timestamp().alias("load_dttm")
    )

    upsert_with_overriding_system_value(
        source_df=f_orders_s3,
        target_table="dwh.f_orders",
        conflict_column="order_id",
        exclude_columns=["product_id"]  # Exclude product_id from updates
    )

    logger.info("ETL Process with OVERRIDING SYSTEM VALUE Completed Successfully!")

except Exception as e:
    logger.error(f"ETL process failed: {e}")
    sys.exit(1)

finally:
    # Stop SparkSession
    spark.stop()
    logger.info("SparkSession stopped.")
